# Programming, Algorithms and Data Structures
Course code: KAN-CDSCO2402U

Name (student-id): Sander Thomasrud (176169)



In [ ]:
from abc import ABC, abstractmethod
from datetime import date, timedelta
import csv
import random
import time
import kagglehub

# Download latest version
path = kagglehub.dataset_download("drahulsingh/best-selling-books")

print("Path to dataset files:", path)

100%|██████████| 6.26k/6.26k [00:00<00:00, 1.95MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/drahulsingh/best-selling-books/versions/2


**Class Book**: represents books that are present in the library

In [ ]:
class Book:

  def __init__(self, isbn, title, author, year, genre, language):
    self._isbn = isbn
    self._title = title
    self._author = author
    self._year = year
    self._genre = genre
    self._language = language
    self._total_rentals = 0
    self._rented = False

  def __str__(self):
    return f"\n{self._title}:\n- ISBN: {self._isbn}\n- Author: {self._author}\n- Year: {self._year}\n- Genre: {self._genre}\n- Language: {self._language}"

  # Sets book as rented and increases total_rentals
  def rent_book(self):
    self._rented = True
    self._total_rentals += 1

  # Sets book as not rented
  def return_book(self):
    self._rented = False

  # Get-methods

  def get_isbn(self):
    return self._isbn

  def get_title(self):
    return self._title

  def get_author(self):
    return self._author

  def get_year(self):
    return self._year

  def get_genre(self):
    return self._genre

  def get_language(self):
    return self._language

  def is_rented(self):
    return self._rented

  def get_total_rentals(self):
    return self._total_rentals

**Class Person**: abstract class that represents all persons in the system

In [ ]:
class Person(ABC):

  def __init__(self, name, age):
    self._name = name
    self._age = age

  @abstractmethod
  def __str__(self):
    return f"Name: {self._name}, Age: {self._age}"

  # Get-methods

  def get_name(self):
    return self._name

  def get_age(self):
    return self._age

**Class Customer**: represents customers in the system; inherits from Person

In [ ]:
class Customer(Person):

  # Sets maximum books per customer at a time
  MAX_BOOKS_RENTED = 5

  def __init__(self, customer_id, name, age):
    super().__init__(name, age)
    self._customer_id = customer_id
    self._books_rented = []
    self._total_rentals = 0

  # Returns boolean for whether customer is eligible for rent or not
  def is_allowed_to_rent(self):
    return len(self._books_rented) < Customer.MAX_BOOKS_RENTED

  # Adds book to rented books list
  def rent_book(self, book):
    self._books_rented.append(book)
    self._total_rentals += 1

  # Removes book from rented books list, if it is there
  def return_book(self, book):
    try:
      self._books_rented.remove(book)
    except:
      print(f"{self.get_name()} has not rented {book.get_title()}, and can therefore not return it.")

  def __str__(self):
    return f"{super().__str__()}\n- Customer ID: {self._customer_id}\n- Books rented: {self._books_rented}"

  # Get-methods

  def get_customer_id(self):
    return self._customer_id

  def get_total_rentals(self):
    return self._total_rentals

  def get_books_rented(self):
    return self._books_rented


**Class Librarian**: represents librarians in the system; inherits from Person

In [ ]:
class Librarian(Person):

  def __init__(self, name, age, employee_id, salary):
    super().__init__(name, age)
    self._employee_id = employee_id
    self._salary = salary


  def __str__(self):
    return f"{super().__str__()}\n- Employee ID: {self._employee_id}\n- Salary: {self._salary}"

  # Get-methods

  def get_employee_id(self):
    return self._employee_id

  def get_salary(self):
    return self._salary

  # Can add see all rentals, see all customers, see rentals per customer, etc


**Class Rental**: represents rentals in the system, with the customer and book

In [ ]:
class Rental:

  def __init__(self, rental_id, customer, book, rental_date, return_date):
    self._rental_id = rental_id
    self._customer = customer
    self._book = book
    self._rental_date = rental_date
    self._return_date = return_date

  # Get-methods

  def get_customer(self):
    return self._customer

  def get_book(self):
    return self._book

  def get_rental_date(self):
    return self._rental_date

  def get_return_date(self):
    return self._return_date

**Class RentalQue**: represents the ques for each book in the system, if there is a que

In [ ]:
class RentalQueue:

  def __init__(self):
    self._queue = {}

  # Adds a customer to the que of a book
  def add_to_queue(self, book, customer):
    if book.get_isbn() in self._queue:
      self._queue[book.get_isbn()].append(customer)
    else:
      self._queue[book.get_isbn()] = [customer]

  # Finds next customer in queue for a book - if there are any in queue
  def find_next_customer(self, book):

    # Tries to get next customer from queue
    try:
      customer = self._queue[book.get_isbn()].pop(0)
      if self._queue[book.get_isbn()] == []:
        del self._queue[book.get_isbn()]
      return customer

    # Raises error if no-one is in queue
    except IndexError:
      print("Error in find_next_customer()")
      return None

  def __str__(self):
    return f"{self._queue}"

  # Get-methods

  def get_queue(self):
    return self._queue


**QuickSort methods**: necessary to perform sorting

In [ ]:

# Main sorting method - takes in an unsorted list of book/customer-objects
def sort(A):

  # Initiates the low and high-limits
  low = 0
  high = len(A) - 1

  # Calls quicksort on the full list
  A = quicksort(A, low, high)

  # Returns the list in a sorted condition
  return A

# Recursive Quicksort method - takes in a list, and the high-low indexes of that list
def quicksort (A, low, high):

    # Returns the list if low-index is same, or higher, than high-index
    if low >= high:
        return A

    # Calls partition-method
    p = partition(A, low, high)

    # Calls itself, on the lower and upper split of the current list
    quicksort(A,low,p-1)
    quicksort(A, p+1, high)

    # Returns list with pivot in the right place
    return A

# Partition method - takes in a list, and its low/high indexes
def partition(A, low, high):
    # Chooses pivot based on list, and high-low indexes
    p = choosepivot(low, high)

    # Swaps pivot element to the right
    A[p], A[high] = A[high], A[p]

    # Initiates pivot-element, left-index, and right-index
    pivot = A[high]
    left = low
    right = high - 1

    # While the left index is smaller or equal to the right
    while left <= right:

        # While the left index is smaller or equal to the right, and element of that index is greater than pivot, increase left index
        while left <= right and A[left].get_total_rentals() >= pivot.get_total_rentals():
            left += 1

        # While the right index is larger or equal to the left, and element of that index is smaller than pivot, decrease right index
        while right >= left and A[right].get_total_rentals() <= pivot.get_total_rentals():
            right -= 1

        # Swaps left and right elements if left index is smaller than right
        if left < right:
            A[left], A[right] = A[right], A[left]

    # Swaps the left element with the pivot
    A[left], A[high] = A[high], A[left]

    # Returns left index - which represents the position with a correct element
    return left

# Pivot selection method - returns pivot index
def choosepivot(low, high):

    return (low + high) // 2

**Class Library**: represents the library system

In [ ]:
class Library:

  def __init__(self):
    self._books = {}
    self._customers = {} # customer_id: customer-object
    self._librarians = []
    self._rentals = {} # rental_id: rental-object
    self._rental_queue = RentalQueue()
    self._isbn_counter = 1
    self._customer_id_counter = 1
    self._rental_id_counter = 1

  # Adds new book to the system
  def add_book(self, new_book):
    self._books[new_book.get_isbn()] = new_book
    self._isbn_counter += 1

  # Adds new customer to the system
  def add_customer(self, name, age):

    # Initiates new Customer-object and adds it to the system
    new_cust = Customer(self._customer_id_counter, name, age)
    self._customers[new_cust.get_customer_id()] = new_cust
    self._customer_id_counter += 1

  # Adds new librarian to the system
  def add_librarian(self, librarian):
    self._librarians.append(librarian)

  # Rents a book in the name of a customer, if book is available and customer is allowed
  def rent_book(self, customer, book):

    # Check if customer has too many books rented
    if not customer.is_allowed_to_rent():
      print(f"{customer.get_name()} has already rented {Customer.MAX_BOOKS_RENTED} books.")

    # Checks if book is rented
    elif book.is_rented():
      print(f"{book.get_title()} is already rented - {customer.get_name()} has been placed in queue.")
      self._rental_queue.add_to_queue(book, customer)

    # Otherwise, proceeds with rental
    else:

      # Finding todays date and return date
      rental_date = date.today()
      return_date = rental_date + timedelta(days=21)

      # Initiating rental object and adding it to rentals
      rental = Rental(self._rental_id_counter, customer, book, rental_date, return_date)
      self._rentals[self._rental_id_counter] = rental

      # Updating the rental id counter
      self._rental_id_counter += 1

      # Set book as rented
      book.rent_book()
      customer.rent_book(book)
      print(f"{book.get_title()} has been rented by {customer.get_name()}.")

  # Returns a book from a customer, if customer has rented the book
  def return_book(self, customer, book):

    # Returns the book from its customer
    customer.return_book(book)
    book.return_book()

    # Checks if there are any customers in queue for the book
    if book.get_isbn() in self._rental_queue.get_queue():

      # Finds a customer, and rents it
      cust = self._rental_queue.find_next_customer(book)
      cust.rent_book(book)
      book.rent_book()

      # Prints message
      print(f"{book.get_title()} has been returned by {customer.get_name()}, and is now rented by {cust.get_name()}.")

    # No customers in queue for this book
    else:
      print(f"{book.get_title()} has been returned by {customer.get_name()}.")


  # Adds book-objects from file
  def add_books_from_file(self, path):

    # Opens file
    with open(path, "r") as file:

        # Use the CSV reader and skip first line
        reader = csv.reader(file)
        next(reader)

        # Iterate the rows
        for row in reader:

            # Unpack the row into variables
            title, author, language, year, sales, genre = row

            # Initiates new Book-object
            book = Book(self._isbn_counter, title, author, year, genre, language)

            # Calls the add_book()-function
            self.add_book(book)

  # Sorts books by total number of rentals - returns sorted list and duration
  def sort_books_by_rentals(self):

    # Starts a timer to find the duration of the sorting
    start_time = time.time()

    # Calls sorting method
    sorted_books = sort(list(self._books.values()))

    # Ends timer
    end_time = time.time()
    sorting_time = end_time - start_time

    # Returns sorted list and duration
    return sorted_books, sorting_time

  # Sorts customers by total number of rentals - returns sorted list and duration
  def sort_customers_by_rentals(self):

    # Starts a timer to find the duration of the sorting
    start_time = time.time()

    # Calls sorting method
    sorted_custs = sort(list(self._customers.values()))

    # Ends timer
    end_time = time.time()
    sorting_time = end_time - start_time

    # Returns sorted list and duration
    return sorted_custs, sorting_time

  # Prints all books in the library
  def print_books(self):
    for book in self._books.values():
      print(book)

  # Get-methods

  def get_book(self, isbn):
    return self._books[isbn]

  def get_customers(self):
    return list(self._customers.values())


**Test Program:**

In [ ]:

# Test program to simulate 100 rentals
def test_rentals():

  # Create a library instance
  library = Library()
  library.add_books_from_file(path + "/best-selling-books.csv")

  # Simulate customers
  customers = [
    ("Alice", 30),
    ("Bob", 25),
    ("Charlie", 35),
    ("Diana", 28),
    ("Eve", 40),
    ("Cora", 22),
    ("Ole", 40),
    ("Ingrid", 15),
    ("Ella", 30),
    ("Ida", 25)
  ]

  # Add customers to the library
  for customer in customers:
    library.add_customer(customer[0], customer[1])

  return_percentage = 0
  # Simulate 200 rentals, with a 70% chance of return for every rental
  for i in range(1, 201):

    # Randomly select a customer
    customer = random.choice(library.get_customers())

    # Randomly select a book by ISBN
    isbn = random.randint(1, len(library._books))
    book = library.get_book(isbn)

    # Attempt to rent the book
    library.rent_book(customer, book)

    # Return a book with 70% chance, if the random customer has an active rental
    if random.random() < 0.7:

      # Pick random customer
      customer = random.choice(library.get_customers())

      try:
        book = random.choice(customer.get_books_rented())
        library.return_book(customer, book)
        return_percentage += 1
      except:
        print(f"{customer.get_name()} has no books rented, and therefore no books are returned.")

  # Display sorted books by total rentals
  sorted_books, books_sorting_time = library.sort_books_by_rentals()
  print("\n--------------------------------------------\n\n\033[1mTop 20 Most Rented Books:\033[0m")

  for book in sorted_books[:20]:
    print(f"{book.get_title()} - Rentals: {book.get_total_rentals()}")

  print(f"\n\033[1mSorting time: {round(books_sorting_time, 4)} seconds\033[0m")

  # Display sorted customers by total rentals
  sorted_customers, customers_sorting_time = library.sort_customers_by_rentals()
  print("\n--------------------------------------------\n\n\033[1mMost active customers:\033[0m")

  for cust in sorted_customers:
    print(f"{cust.get_name()} - Rentals: {cust.get_total_rentals()}")

  print(f"\n\033[1mSorting time: {round(customers_sorting_time, 4)} seconds\033[0m")
  print(f"\n--------------------------------------------\n\n\033[1mReturn percentage: {round(return_percentage / 200 * 100, 1)}%\033[0m")
# Run the test program
test_rentals()



Harry Potter and the Philosopher's Stone has been rented by Ingrid.
Cora has no books rented, and therefore no books are returned.
The Grapes of Wrath has been rented by Charlie.
Guess How Much I Love You has been rented by Ole.
Bob has no books rented, and therefore no books are returned.
Shōgun has been rented by Ole.
Ida has no books rented, and therefore no books are returned.
The Young Guard (Молодая гвардия) has been rented by Ida.
Ella has no books rented, and therefore no books are returned.
Fifty Shades Darker has been rented by Cora.
Bob has no books rented, and therefore no books are returned.
The Stranger (L'Étranger) has been rented by Charlie.
Perfume (Das Parfum) has been rented by Cora.
The Grapes of Wrath has been returned by Charlie.
Jaws has been rented by Ingrid.
The Exorcist has been rented by Ida.
Totto-chan, the Little Girl at the Window (窓ぎわのトットちゃん) has been rented by Eve.
Perfume (Das Parfum) has been returned by Cora.
The Secret Diary of Adrian Mole, Aged 13¾ 